##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Prompting Quickstart

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Prompting.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

This notebook contains examples of how to write and run your first prompts with the Gemini API.

In [1]:
%pip install -U -q "google-genai>=1.4.0" # 1.4.0 is needed for chat history

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.7/159.7 kB 3.9 MB/s eta 0:00:00


### Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [2]:
from google.colab import userdata
from google import genai

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

## Select your model

Now select the model you want to use in this guide, either by selecting one in the list or writing it down. Keep in mind that some models, like the 2.5 ones are thinking models and thus take slightly more time to respond (cf. [thinking notebook](./Get_started_thinking.ipynb) for more details and in particular learn how to switch the thiking off).

In [3]:
MODEL_ID = "gemini-2.5-flash-preview-04-17" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-04-17","gemini-2.5-pro-exp-03-25"] {"allow-input":true, isTemplate: true}

## Run your first prompt

Use the `generate_content` method to generate responses to your prompts. You can pass text directly to generate_content, and use the `.text` property to get the text content of the response.

In [4]:
from IPython.display import Markdown

response = client.models.generate_content(
    model=MODEL_ID,
    contents="Give me python code to sort a list"
)

display(Markdown(response.text))

Okay, Python provides two primary ways to sort a list:

1.  **`list.sort()`:** This method sorts the list **in-place**. It modifies the original list directly and returns `None`.
2.  **`sorted(list)`:** This is a built-in function that returns a **new** sorted list. It does **not** modify the original list.

Here are examples of how to use both:

**1. Using `list.sort()` (Sorts In-Place)**

```python
# Define a list
my_list = [3, 1, 4, 1, 5, 9, 2, 6]

# Sort the list in-place (modifies the original list)
my_list.sort()

# Print the modified list
print("Using list.sort():", my_list)

# Example with strings
string_list = ["banana", "apple", "cherry", "date"]
string_list.sort()
print("Using list.sort() on strings:", string_list)

# Example of sorting in descending order
my_list_desc = [3, 1, 4, 1, 5, 9, 2, 6]
my_list_desc.sort(reverse=True)
print("Using list.sort() (descending):", my_list_desc)
```

**2. Using `sorted(list)` (Returns a New List)**

```python
# Define a list (original remains unchanged)
my_list = [3, 1, 4, 1, 5, 9, 2, 6]

# Get a new sorted list using the sorted() function
new_sorted_list = sorted(my_list)

# Print the original list (it's unchanged)
print("Original list:", my_list)

# Print the new sorted list
print("Using sorted():", new_sorted_list)

# Example with strings
string_list = ["banana", "apple", "cherry", "date"]
new_sorted_string_list = sorted(string_list)
print("Using sorted() on strings:", new_sorted_string_list)
print("Original string list (unchanged):", string_list) # Show original is unchanged

# Example of sorting in descending order
my_list_desc = [3, 1, 4, 1, 5, 9, 2, 6]
new_desc_sorted_list = sorted(my_list_desc, reverse=True)
print("Using sorted() (descending):", new_desc_sorted_list)
```

**Key Difference:**

*   Choose `list.sort()` when you don't need the original list and want to save memory by modifying it directly.
*   Choose `sorted(list)` when you need to keep the original list intact and get a sorted copy.

Both methods can accept a `key` argument for custom sorting criteria (e.g., sorting a list of strings by their length).

## Use images in your prompt

Here you will download an image from a URL and pass that image in our prompt.

First, you download the image and load it with PIL:

In [5]:
!curl -o image.jpg "https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  349k  100  349k    0     0  2216k      0 --:--:-- --:--:-- --:--:-- 2224k


In [6]:
import PIL.Image
img = PIL.Image.open('image.jpg')
img

Output hidden; open in https://colab.research.google.com to view.

In [7]:
prompt = """
    This image contains a sketch of a potential product along with some notes.
    Given the product sketch, describe the product as thoroughly as possible based on what you
   see in the image, making sure to note all of the product features. Return output in json format:
   {description: description, features: [feature1, feature2, feature3, etc]}
"""

Then you can include the image in our prompt by just passing a list of items to `generate_content`.

In [11]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[prompt, img],
)

print(response.text)

```json
{
  "description": "The Jetpack Backpack is a product designed to resemble a normal backpack while incorporating jetpack capabilities. It features a padded strap support for comfort, a compartment large enough to fit an 18-inch laptop, and retractable boosters located at the bottom. The jetpack is described as lightweight, steam-powered, and environmentally friendly (green/clean). It offers USB-C charging and has a stated battery life of 15 minutes.",
  "features": [
    "Fits 18\" laptop",
    "Padded strap support",
    "Lightweight",
    "Looks like a normal backpack",
    "USB-C charging",
    "15-min battery life",
    "Retractable boosters",
    "Steam-powered",
    "Green/clean (environmentally friendly)"
  ]
}
```


## Have a chat

The Gemini API enables you to have freeform conversations across multiple turns.

The [ChatSession](https://ai.google.dev/api/python/google/generativeai/ChatSession) class will store the conversation history for multi-turn interactions.

In [12]:
chat = client.chats.create(model=MODEL_ID)

In [13]:
response = chat.send_message(
    message="In one sentence, explain how a computer works to a young child."
)

print(response.text)

A computer is like a super-fast helper that follows tiny instructions to show you pictures, play games, and help you do things.


You can see the chat history:

In [14]:
messages = chat.get_history()
for message in messages:
  print(f"{message.role}: {message.parts[0].text}")

user: In one sentence, explain how a computer works to a young child.
model: A computer is like a super-fast helper that follows tiny instructions to show you pictures, play games, and help you do things.


You can keep sending messages to continue the conversation:

In [17]:
response = chat.send_message("Okay, how about a more detailed explanation to a high schooler?")

print(response.text)

Okay, for a high schooler:

A computer works by taking input data (from your keyboard, mouse, etc.), processing it at incredibly high speeds using its central processing unit (CPU) based on detailed step-by-step instructions from programs (software) held temporarily in fast memory (RAM), and then producing output (like showing things on your screen or saving files to storage) according to those instructions.


## Set the temperature

Every prompt you send to the model includes parameters that control how the model generates responses. Use a `types.GenerateContentConfig` to set these, or omit it to use the defaults.

Temperature controls the degree of randomness in token selection. Use higher values for more creative responses, and lower values for more deterministic responses.

Note: Although you can set the `candidate_count` in the generation_config, 2.0 and later models will only return a single candidate at the this time.

In [22]:
from google.genai import types

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Give me a numbered list of cat facts.',
    config=types.GenerateContentConfig(
        max_output_tokens=2000,
        temperature=1.9,
        stop_sequences=['\n6'] # Limit to 5 facts.
    )
)

display(Markdown(response.text))

Okay, here's a list of fascinating cat facts:

1.  **Cats sleep a lot:** On average, cats sleep for 12-16 hours a day.

2.  **Unique nose prints:** Like human fingerprints, a cat's nose has a unique pattern of ridges and bumps.

3.  **Whiskers are multi-functional:** Whiskers are not just cute; they help cats navigate in the dark, sense changes in air currents, and even gauge the width of openings.

4.  **Purring is complex:** While cats often purr when content, they can also purr when stressed, injured, or giving birth as a form of self-soothing. The frequency (25-150 Hertz) can promote healing.

5.  **Excellent jumpers:** Cats can jump up to six times their height thanks to their strong leg muscles and flexible spine.


## Learn more

There's lots more to learn!

* For more fun prompts, check out [Market a Jetpack](https://github.com/google-gemini/cookbook/blob/main/examples/Market_a_Jet_Backpack.ipynb).
* Check out the [safety quickstart](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Safety.ipynb) next to learn about the Gemini API's configurable safety settings, and what to do if your prompt is blocked.
* For lots more details on using the Python SDK, check out the [get started notebook](./Get_started.ipynb) or the [documentation's quickstart](https://ai.google.dev/tutorials/python_quickstart).